In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
from complexPyTorch.complexLayers import  ComplexLinear
import torch
import pennylane as qml
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
dev = qml.device('default.qubit', wires=6)
@qml.qnode(dev)
def circuit(weights,input):
    weights = weights.view(-1, 6, 3)
    qml.AmplitudeEmbedding(features=input,normalize=True, wires=[0,1,2,3,4,5], pad_with=0)
    qml.Barrier(wires=range(6), only_visual=True)
    qml.StronglyEntanglingLayers(weights,wires=range(6))
    return  qml.state()

In [ ]:
class  ComplexNet(nn.Module):
    def __init__(self):
        super(ComplexNet,self).__init__()
        self.weights = nn.Parameter(torch.randn(1, 6*3), requires_grad=True)
        self.com_linear1 =ComplexLinear(64, 128)
        self.com_linear2 =ComplexLinear(128, 64)
        self.com_linear3 =ComplexLinear(64, 32)
        self.com_linear4 =ComplexLinear(32,5)
    @staticmethod
    def ovonic_complex_relu(input):
        zero = torch.tensor(0)
        return torch.maximum(input.real, zero).type(torch.complex64) \
               + 1j * torch.maximum(input.imag, zero).type(torch.complex64)
    def forward(self, x):
        x = circuit(self.weights, x).type(torch.complex64)
        x = self.ovonic_complex_relu(self.com_linear1(x))
        x = self.ovonic_complex_relu(self.com_linear2(x))
        x = self.ovonic_complex_relu(self.com_linear3(x))
        x = self.com_linear4(x)
        x = (F.log_softmax(x.real, dim=1) +  F.log_softmax(x.imag, dim=1))/2
        return x

In [ ]:
k=10
epochs = 500
batch_size =64
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

with open("QCVNN_training_metrics.txt", "w") as file:
    file.write("Fold\tEpoch\tTrain Accuracy\tTrain Loss\tVal Accuracy\tVal Loss\t\tTest Accuracy\tVal Loss\n")

for fold in range(k):
    train_dataset = pd.read_csv(r'X_train_fold_'+str(fold)+'.csv')
    val_dataset = pd.read_csv(r'X_val_fold_'+str(fold)+'.csv')
    test_dataset = pd.read_csv(r'X_test.csv')

    train_elements_array = np.array(train_dataset.iloc[:,4:47])
    train_encoded_labels = np.array(train_dataset['labels_encoder'])
    val_elements_array = np.array(val_dataset.iloc[:,4:47])
    val_encoded_labels = np.array(val_dataset['labels_encoder'])
    test_elements_array = np.array(test_dataset.iloc[:,4:47])
    test_encoded_labels  = np.array(test_dataset['labels_encoder'])

    train_set = torch.tensor(train_elements_array, dtype=torch.float32)
    train_label = torch.tensor(train_encoded_labels, dtype=torch.long)
    train_label_dataset = TensorDataset(train_set, train_label)
    train_loader = DataLoader(train_label_dataset, batch_size=batch_size, shuffle=True)

    val_set = torch.tensor(val_elements_array, dtype=torch.float32)
    val_label = torch.tensor(val_encoded_labels, dtype=torch.long)
    val_label_dataset = TensorDataset(val_set, val_label)
    val_loader = DataLoader(val_label_dataset, batch_size=batch_size, shuffle=False)

    test_set = torch.tensor(test_elements_array, dtype=torch.float32)
    test_label = torch.tensor(test_encoded_labels, dtype=torch.long)
    test_label_dataset = TensorDataset(test_set, test_label)
    test_loader = DataLoader(test_label_dataset, batch_size=batch_size, shuffle=False)

    model = ComplexNet().to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.95)

    train_loss_list = []
    train_accuracy_list= []
    test_loss_list = []
    test_accuracy_list= []
    best_test_acc = 0
    best_test_loss = 0
    for epoch in range(epochs):
        train_loss = 0
        train_accuracy= 0
        avg_train_accuracy = 0
        avg_train_loss = 0
        test_loss = 0
        test_accuracy= 0
        avg_test_accuracy = 0
        avg_test_loss = 0

        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            # 计算预测结果
            preds = torch.argmax(output, dim=1)
            # 计算准确率
            train_accuracy += (preds == target).sum()
            train_loss +=loss.item()
            loss.backward()
            optimizer.step()

        avg_train_accuracy = train_accuracy / len(train_loader.sampler)
        avg_train_loss = train_loss/ len(train_loader.sampler)
        train_accuracy_list.append(avg_train_accuracy)
        train_loss_list.append(avg_train_loss)

        model.eval()
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = F.nll_loss(output, target)
            preds = torch.argmax(output, dim=1)

            test_accuracy += (preds == target).sum()
            test_loss +=loss.item()
        avg_test_accuracy = test_accuracy / len(test_loader.sampler)
        avg_test_loss = test_loss/ len(test_loader.sampler)
        test_accuracy_list.append(avg_test_accuracy)
        test_loss_list.append(avg_test_loss)

        if best_test_acc < avg_test_accuracy:
            best_test_acc = avg_test_accuracy
            best_test_loss = avg_test_loss
            if epoch >= epochs-100 or best_test_acc>0.94:
                print('save best model best_test_acc',best_test_acc)
                torch.save(model.state_dict(), 'QCVNN_model/model_test_fold_'+str(fold)+'.pth')
        print('Fold {:3} Train Epoch:{:3}  accuracy: {:.4f}  Loss: {:.4f}  test accuracy: {:.4f}  Loss: {:.4f}'.format(
                    fold,
                    epoch,
                    avg_train_accuracy,
                    avg_train_loss,
                    avg_test_accuracy,
                    avg_test_loss
                ))
         # 将指标写入txt文件
        with open("QCVNN_training_metrics.txt", "a") as file:
            file.write("{:3}\t{:3}\t{:.4f}\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}\n".format(
                fold, epoch, avg_train_accuracy, avg_train_loss,  avg_test_accuracy, avg_test_loss))
    print("Fold [%d]  Best_test_Accuracy: %.4f Best_test_Loss: %.4f" % (fold,  best_test_acc, best_test_loss))

In [ ]:
from sklearn.metrics import  f1_score, recall_score, precision_score
from sklearn.metrics import confusion_matrix
import torch

k=10
cm_list=[]
acc_list = []
f1_list = []
recall_list = []
precision_list = []
class_accuracy_list =[]

cm_list_val=[]
acc_list_val = []
f1_list_val = []
recall_list_val = []
precision_list_val = []
class_accuracy_list_val = []
for fold in range(k):


    val_dataset = pd.read_csv(r'X_val_fold_'+str(fold)+'.csv')
    val_elements_array = np.array(val_dataset.iloc[:,4:47])
    val_encoded_labels = np.array(val_dataset['labels_encoder'])



    model = ComplexNet()
    model.load_state_dict(torch.load('QCVNN_model.pth'))
    model.eval()

    output = model.forward(val_elements_array)
    y_pred = torch.argmax(output, dim=1)
     # 计算分类器的准确率
    accuracy_val = accuracy_score(val_encoded_labels, y_pred)
    f1_val = f1_score(val_encoded_labels, y_pred,average='weighted')
    recall_val = recall_score(val_encoded_labels, y_pred,average='weighted')
    precision_val = precision_score(val_encoded_labels, y_pred,average='weighted')
    cm_val = confusion_matrix(val_encoded_labels, y_pred)

    acc_list_val.append(accuracy_val)
    f1_list_val.append(f1_val)
    recall_list_val.append(recall_val)
    precision_list_val.append(precision_val)
    cm_list_val.append(cm_val)
    # 计算每个类别的准确率
    class_accuracy = []
    for class_label in range(5):  # num_classes 是类别的数量
        # 选出属于当前类别的样本
        class_indices = (val_encoded_labels == class_label)
        class_samples = val_elements_array[class_indices]
        class_labels = val_encoded_labels[class_indices]
        # 对当前类别的样本进行预测
        class_output = model.forward(class_samples)
        class_y_pred = torch.argmax(class_output, dim=1)
        # 计算当前类别的准确率
        class_accuracy.append( accuracy_score(class_labels, class_y_pred))

    print("每个类别的准确率:", class_accuracy)
    class_accuracy_list.append(class_accuracy)
    print("Fold [%d]  Val Accuracy: %.4f  F1: %.4f Recall: %.4f Precision: %ds.4f" % (fold, accuracy_val,  f1_val, recall_val, precision_val))

mean_cm_val = np.array(np.sum(cm_list_val, axis=0) /10)

mean_acc_val =  np.mean(acc_list_val)
mean_f1_val=  np.mean(f1_list_val)
mean_recall_val=   np.mean(recall_list_val)
mean_precision_val= np.mean(precision_list_val)
mean_class_acc= np.mean(class_accuracy_list,axis=0)


print("All Fold: Val Mean Accuracy: %.4f  Mean F1: %.4f  Mean Recall: %.4f Mean Precision: %.4f Mean confusion_matrix: %.4f" % (mean_acc_val, mean_f1_val, mean_recall_val,mean_precision_val,mean_cm_val ))